In [4]:
import os
from pathlib import Path

In [5]:
!pwd

/mnt/c/Users/FARHAN/Desktop/Content/Python-for-everyone/Complete_MLOps_Project/experiments


In [6]:
os.chdir('../')

In [7]:
!pwd

/mnt/c/Users/FARHAN/Desktop/Content/Python-for-everyone/Complete_MLOps_Project


In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [17]:
!pip install joblib ensure python-box==6.0.2

Defaulting to user installation because normal site-packages is not writeable
  Using cached python_box-6.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)


In [18]:
from src.config.configuration import DataIngestionConfig
from src.utils.common import create_directories, read_yaml
from src.constants import *

In [23]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([Path(self.config.artifacts_root)])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([Path(config.root_dir)])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [24]:
import os
import urllib.request as request
import zipfile
from src.logger.logging import Logger
from src.utils.common import get_size

In [25]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            Logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            Logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [26]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-08-06 21:13:35,265]: yaml file : config/config.yml loaded succesfully.
[2024-08-06 21:13:35,288]: yaml file : params.yml loaded succesfully.
[2024-08-06 21:13:35,304]: Directory : artifacts created.
[2024-08-06 21:13:35,317]: Directory : artifacts/data_ingestion created.
[2024-08-06 21:13:35,330]: File already exists of size: 11.08 MB
